In [1]:
import pandas as pd
import pickle

import ipywidgets as widgets
from ipywidgets import interact

from src.data.data_load import (
    load_tables, 
    load_online_instance, 
    load_distances, 
    upload_ONLINE_static_solution
)
from src.data.solution_load import load_solution_dfs
from src.utils.filtering import flexible_filter
from src.utils.plotting import plot_metrics_comparison_dynamic
from src.data.metrics import collect_results_to_df, compute_metrics_with_moves, get_day_plotting_df
from src.config.experimentation_config import *
from src.config.config import *

data_path = '../data'

distance_type = 'osrm'              # Options: ['osrm', 'manhattan']
dist_method = 'haversine'      # Options: ['precalced', 'haversine']

optimization_obj = 'driver_distance'

directorio_df, labors_raw_df, cities_df, duraciones_df, valid_cities = load_tables(data_path, generate_labors=False)
# labors_real_df, labors_static_df, labors_dynamic_df = load_online_instance(data_path, instance, labors_raw_df)
# dist_dict = load_distances(data_path, distance_type, instance, dist_method)

metricas = ['service_count', 'vt_count', 'num_drivers', 'driver_extra_time', 'driver_move_distance']

# Instance exploration

In [2]:
instance = 'N500/normal/seed_0'
labors_real_df, labors_static_df, labors_dynamic_df = load_online_instance(data_path, instance, labors_raw_df)


In [3]:
labors_real_df

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,city,address_id,address_point,address_name,created_date_only,schedule_date_only,is_static,is_dynamic,simulated_day,date
0,248341,343990,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,55663.0,2025-05-29 15:05:11.639000-05:00,2026-11-10 15:52:00-05:00,2026-11-10 19:28:00-05:00,21405,...,1,28134.0,POINT (-75.6004631859725 6.254282787660095),otros,2025-05-29,2025-06-18,True,False,2026-11-11,2026-11-11
1,259351,355791,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,49485.0,2025-06-24 17:07:47.975000-05:00,2026-11-10 10:16:00-05:00,2026-11-10 11:39:00-05:00,5455,...,1,10331.0,POINT (-75.5568613 6.278588999999999),Casa,2025-06-24,2025-06-26,True,False,2026-11-11,2026-11-11
2,254899,351054,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,55663.0,2025-06-13 15:10:50.613000-05:00,2026-11-10 16:23:00-05:00,2026-11-10 17:52:00-05:00,2036,...,1,55748.0,POINT (-75.55748439999999 6.252939599999999),Apartamento,2025-06-13,2025-06-25,True,False,2026-11-11,2026-11-11
3,254376,350497,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,55663.0,2025-06-12 15:14:36.360000-05:00,2026-11-10 07:16:00-05:00,2026-11-10 09:24:00-05:00,2036,...,1,55748.0,POINT (-75.55748439999999 6.252939599999999),Apartamento,2025-06-12,2025-06-16,True,False,2026-11-11,2026-11-11
4,249774,345521,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,55663.0,2025-06-03 08:13:41.011000-05:00,2026-11-10 07:47:00-05:00,2026-11-10 10:07:00-05:00,21405,...,1,28134.0,POINT (-75.6004631859725 6.254282787660095),otros,2025-06-03,2025-06-04,True,False,2026-11-11,2026-11-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,257762,354107,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,55663.0,2025-06-19 17:19:11.997000-05:00,2026-11-10 06:45:00-05:00,2026-11-10 07:41:00-05:00,6412,...,844,12312.0,POINT (-73.1037471 7.0910492),Apartamento,2025-06-19,2025-06-20,True,False,2026-11-11,2026-11-11
496,259123,355549,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,55663.0,2025-06-24 10:23:02.543000-05:00,2026-11-10 15:47:00-05:00,2026-11-10 16:52:00-05:00,6412,...,844,12312.0,POINT (-73.1037471 7.0910492),Apartamento,2025-06-24,2025-06-24,False,True,2026-11-11,2026-11-11
497,253069,349168,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,90004.0,2025-06-10 14:52:25.187000-05:00,2026-11-10 14:54:00-05:00,2026-11-10 15:51:00-05:00,6412,...,844,12312.0,POINT (-73.1037471 7.0910492),Apartamento,2025-06-10,2025-06-10,False,True,2026-11-11,2026-11-11
498,253583,349629,6.0,General Mechanics,GENERAL_MECHANICS,1178100.0,2025-06-11 09:59:57.845000-05:00,2026-11-10 14:38:00-05:00,2026-11-15 10:44:00-05:00,,...,844,107054.0,POINT (-73.1287071 7.132651399999999),Taller Principal Motors,2025-06-11,2025-06-11,False,True,2026-11-11,2026-11-11


In [4]:
flexible_filter(
    labors_real_df,
    service_id='251555')
                

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,city,address_id,address_point,address_name,created_date_only,schedule_date_only,is_static,is_dynamic,simulated_day,date
8,251555,347443,3.0,Wash and Polish,WASH_AND_POLISH,39270.0,2025-06-06 14:03:58.921000-05:00,2026-11-10 10:58:38-05:00,2026-11-10 11:46:30-05:00,,...,1,39882.0,POINT (-75.5692949 6.166495299999999),Auto Spa La Sebastiana,2025-06-06,2025-06-07,True,False,2026-11-11,2026-11-11


In [5]:
flexible_filter(
    labors_raw_df,
    service_id='251555')
                

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,paying_customer,state_service,start_address_id,start_address_point,end_address_id,end_address_point,city,address_id,address_point,address_name
99151,251555,347442,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,35000.0,2025-06-06 14:03:58.853000-05:00,2025-06-07 09:09:10-05:00,2025-06-07 10:58:38-05:00,20111,...,64682.0,COMPLETED,112710.0,POINT (-75.6120011 6.171409799999999),112710.0,POINT (-75.6120011 6.171409799999999),1,25905.0,POINT (-75.594906 6.190818699999999),Apto
99152,251555,347443,3.0,Wash and Polish,WASH_AND_POLISH,39270.0,2025-06-06 14:03:58.921000-05:00,2025-06-07 10:58:38-05:00,2025-06-07 11:46:30-05:00,,...,64682.0,COMPLETED,112710.0,POINT (-75.6120011 6.171409799999999),112710.0,POINT (-75.6120011 6.171409799999999),1,39882.0,POINT (-75.5692949 6.166495299999999),Auto Spa La Sebastiana
99153,251555,347444,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,35000.0,2025-06-06 14:03:58.976000-05:00,2025-06-07 11:46:30-05:00,2025-06-07 13:25:28-05:00,20111,...,64682.0,COMPLETED,112710.0,POINT (-75.6120011 6.171409799999999),112710.0,POINT (-75.6120011 6.171409799999999),1,25905.0,POINT (-75.594906 6.190818699999999),Apto
